In [1]:
####Visualizations
#### Importing dependencies 
import numpy as np
import pandas as pa
import seaborn as se
import matplotlib.pyplot as plt
from IPython.display import HTML

In [176]:
##### Data preparation for the Prediction.


ball_by_ball=pa.read_csv("./train/train/ball_by_ball_data.csv")
player_roster=pa.read_csv("./train/train/player_rosters.csv")
key_teams=pa.read_csv('./train/train/key_teams.csv')
match_data=pa.read_csv('./train/train/match_data.csv')
players_2018 = pa.read_csv("./test/test/player_predictions.csv")
matchs_2018 = pa.read_csv("./test/test/matches_2018.csv")


In [74]:
##### Now take ball_by_ball data for preprocessing.
##### Grouping the data such a way that we get the runs scored and wickets taken by each player for each match.
##### One more constraint needed for grouping is innings based on innings we are grouping and splitting into two datasets.


batsman_score=ball_by_ball[['match_id','inning','batting_team','bowling_team','batsman','batsman_id','batsman_runs']]
batsman_score.head()
batsman_score=batsman_score.groupby(['match_id','inning','batting_team','bowling_team','batsman','batsman_id'], as_index=False)['batsman_runs'].sum()
batsman_score=batsman_score.merge((match_data[['match_id','venue_id']]), on='match_id')


team_codes={'CSK':0, 'DC':1, 'DD':2, 'GL':3, 'KKR':4, 'KTK':5, 'KXI':6, 'MI':7, 'PW':8, 'RCB':9,'RPS':10, 'RR':11, 'SRH':12}
batsman_score=batsman_score.replace({'batting_team':team_codes,'bowling_team':team_codes})


batsman_score_1st_inn=batsman_score[batsman_score['inning']==1]
batsman_score_2nd_inn=batsman_score[batsman_score['inning']==2]


In [75]:
wickets_taken = ball_by_ball[['match_id','inning','batting_team','bowling_team','bowler','bowler_id','dismissal_kind']]

In [76]:
wickets_taken.fillna(0,inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [77]:
wickets_taken.head()

,match_id,inning,batting_team,bowling_team,bowler,bowler_id,dismissal_kind
0,2017_1,1,SRH,RCB,Tymal Mills,621,0
1,2017_1,1,SRH,RCB,Tymal Mills,621,0
2,2017_1,1,SRH,RCB,Tymal Mills,621,0
3,2017_1,1,SRH,RCB,Tymal Mills,621,0
4,2017_1,1,SRH,RCB,Tymal Mills,621,0


In [8]:
wickets_taken['dismissal_kind'].unique()

array([0, 'caught', 'bowled', 'run out', 'lbw', 'caught and bowled',
       'stumped', 'retired hurt', 'hit wicket', 'obstructing the field'], dtype=object)

In [78]:
dismissal_counted_for_bowlers=['caught', 'bowled', 'lbw', 'caught and bowled','stumped','hit wicket']

In [79]:
mapping={'caught':1, 'bowled':1, 'lbw':1, 'caught and bowled':1,'stumped':1,'hit wicket':1,'run out':0,'retired hurt':0,'obstructing the field':0}

In [80]:
wickets_taken=wickets_taken.replace({'dismissal_kind':mapping,'batting_team':team_codes,'bowling_team':team_codes})

In [81]:
wickets_taken=wickets_taken.groupby(['match_id','inning','batting_team','bowling_team','bowler','bowler_id'],as_index=False)['dismissal_kind'].sum()

In [82]:
wickets_taken=wickets_taken.merge((match_data[['match_id','venue_id']]), on='match_id')

In [83]:
wickets_taken_1st_inn=wickets_taken[wickets_taken['inning']==1]
wickets_taken_2nd_inn=wickets_taken[wickets_taken['inning']==2]

In [91]:
batsman_score_1st_inn

,match_id,inning,batting_team,bowling_team,batsman,batsman_id,batsman_runs,venue_id
0,2008_1,1,4,9,Brendon McCullum,82,158,7
1,2008_1,1,4,9,David Hussey,80,12,7
2,2008_1,1,4,9,Mohammad Hafeez,83,5,7
3,2008_1,1,4,9,Ricky Ponting,73,20,7
4,2008_1,1,4,9,Sourav Ganguly,66,10,7
16,2008_10,1,6,7,Brett Lee,94,16,9
17,2008_10,1,6,7,Irfan Pathan,96,6,9
18,2008_10,1,6,7,Karan Goel,104,18,9
19,2008_10,1,6,7,Kumar Sangakkara,98,94,9
20,2008_10,1,6,7,Mahela Jayawardene,101,12,9


###########################################################################################################

In [185]:
palyer_palying_in_2018=np.unique(players_2018[['player_id']])

In [189]:
##### Selecting the only players that are playing in 2018.

batsman_score=batsman_score[batsman_score['batsman_id'].isin(palyer_palying_in_2018)]

In [190]:
batsman_score_1st_inn=batsman_score[batsman_score['inning']==1]
batsman_score_2nd_inn=batsman_score[batsman_score['inning']==2]

In [191]:
palyer_palying_in_2018

array([  1,   3,  12,  24,  30,  47,  52,  55,  57,  58,  59,  63,  71,
        79,  82,  84,  88, 102, 114, 118, 120, 122, 126, 131, 141, 149,
       153, 157, 171, 175, 177, 189, 207, 208, 211, 249, 250, 254, 271,
       305, 309, 311, 315, 321, 322, 333, 337, 340, 341, 343, 351, 359,
       361, 373, 378, 386, 402, 411, 412, 429, 430, 433, 437, 439, 443,
       449, 456, 458, 464, 465, 467, 469, 471, 478, 479, 480, 484, 485,
       487, 500, 502, 507, 511, 512, 514, 518, 521, 526, 528, 529, 531,
       535, 537, 540, 548, 551, 553, 559, 562, 564, 565, 567, 570, 571,
       572, 573, 574, 581, 582, 590, 593, 594, 596, 598, 600, 606, 608,
       610, 613, 614, 617, 618, 619, 622, 625, 626, 630, 631, 632, 633,
       634, 635, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647,
       648, 649, 650, 651, 652, 654, 655, 656, 657, 658, 659, 661, 662,
       663, 664, 665, 666, 668, 669, 670, 671, 672, 673, 674, 675, 676,
       677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 68

In [192]:
pa.DataFrame(columns=palyer_palying_in_2018)

,1,3,12,24,30,47,52,55,57,58,...,685,686,687,688,689,690,691,692,693,696


In [198]:
previous_players=list(pa.get_dummies(batsman_score['batsman_id']).columns)

In [222]:
ipl_2018=players_2018[['match_id','player_id','team_id']]

In [208]:
ipl_2018

,match_id,player_id,team_id
0,2018_1,1,CSK
1,2018_1,24,CSK
2,2018_1,114,CSK
3,2018_1,131,CSK
4,2018_1,175,CSK
5,2018_1,177,CSK
6,2018_1,189,CSK
7,2018_1,249,CSK
8,2018_1,254,CSK
9,2018_1,309,CSK


In [223]:
ipl_2018_final=ipl_2018.merge((matchs_2018[['match_id','team1_id','team2_id','venue_id']]),on='match_id')

In [224]:
ipl_2018_final['bowling_team']='NaN'

In [239]:
ipl_2018_final

,match_id,player_id,batting_team,venue_id,bowling_team,batsman_runs
0,2018_1,1,CSK,1,MI,NaN
1,2018_1,24,CSK,1,MI,NaN
2,2018_1,114,CSK,1,MI,NaN
3,2018_1,131,CSK,1,MI,NaN
4,2018_1,175,CSK,1,MI,NaN
5,2018_1,177,CSK,1,MI,NaN
6,2018_1,189,CSK,1,MI,NaN
7,2018_1,249,CSK,1,MI,NaN
8,2018_1,254,CSK,1,MI,NaN
9,2018_1,309,CSK,1,MI,NaN


In [227]:
for i in range(len(ipl_2018_final)):
    if ipl_2018_final.iloc[i,2]!=ipl_2018_final.iloc[i,3]:
        ipl_2018_final.iloc[i,6]=ipl_2018_final.iloc[i,3]
    else:
        ipl_2018_final.iloc[i,6]=ipl_2018_final.iloc[i,4]

In [231]:
del ipl_2018_final['team1_id']
del ipl_2018_final['team2_id']

In [250]:
ipl_2018_final=ipl_2018_final.rename(columns={'team_id':'batting_team','player_id':'batsman_id'})

In [238]:
ipl_2018_final['batsman_runs']='NaN'

In [245]:
ipl_2018_final['inning']='NaN'

In [242]:
ipl_2018_final=ipl_2018_final.replace({'batting_team':team_codes,'bowling_team':team_codes})

In [253]:
ipl_2018_final=ipl_2018_final[['match_id','inning','batting_team','bowling_team','batsman_id','venue_id','batsman_runs']]

In [254]:
batsman_score=batsman_score[['match_id','inning','batting_team','bowling_team','batsman_id','venue_id','batsman_runs']]

In [259]:
combined_data=pa.concat([batsman_score,ipl_2018_final])

In [293]:
batting_players_dummy=pa.get_dummies(combined_data['batsman_id'])

In [264]:
combined_data['batting_team']="batting_"+combined_data['batting_team'].apply(str)

In [265]:
combined_data['bowling_team']="bowling_"+combined_data['bowling_team'].apply(str)

In [273]:
combined_data['venue_id']="venue_id_"+combined_data['venue_id'].apply(str)

In [290]:
batting_dummy_variables=pa.get_dummies(combined_data['batting_team'])

In [270]:
pa.get_dummies(combined_data['bowling_team'])

,bowling_0,bowling_1,bowling_10,bowling_11,bowling_12,bowling_2,bowling_3,bowling_4,bowling_5,bowling_6,bowling_7,bowling_8,bowling_9
0,0,0,0,0,0,0,0,0,0,0,0,0,1
13,0,0,0,0,0,0,0,1,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,0,1,0,0
26,0,0,0,0,0,0,0,0,0,0,1,0,0
29,0,0,0,0,0,0,0,0,0,1,0,0,0
30,0,0,0,0,0,0,0,0,0,1,0,0,0
31,0,0,0,0,0,0,0,0,0,1,0,0,0
34,0,0,0,0,0,0,0,0,0,1,0,0,0
36,0,0,0,0,0,0,0,0,0,1,0,0,0
45,0,0,0,1,0,0,0,0,0,0,0,0,0


In [283]:
venuues=pa.get_dummies(combined_data['venue_id'])

In [289]:
##### combining venues dummy variables.
final_combined_data=pa.concat([combined_data,venuues],axis=1)

In [292]:
#### combinng batting teams dummy variables.
final_combined_data=pa.concat([final_combined_data,batting_dummy_variables],axis=1)

In [294]:
#### Combining player dummy varibles.

final_combined_data=pa.concat([final_combined_data,batting_players_dummy],axis=1)

In [304]:
final_combined_data=final_combined_data.reset_index()


In [306]:
##### Removing the venue_id bowling, batting team, batsman_id also.

del final_combined_data['batting_team']
del final_combined_data['bowling_team']
del final_combined_data['batsman_id']
del final_combined_data['venue_id']
del final_combined_data['index']


In [307]:
final_combined_data

,match_id,inning,batsman_runs,venue_id_1,venue_id_10,venue_id_11,venue_id_12,venue_id_13,venue_id_14,venue_id_15,...,685,686,687,688,689,690,691,692,693,696
0,2008_1,1,158,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2008_1,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2008_10,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2008_10,1,18,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2008_10,2,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2008_10,2,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2008_10,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2008_10,2,21,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2008_10,2,17,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2008_11,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [322]:
#### now let me seperate the 2018 matches since we use it for prediction.

featured_2018_data=final_combined_data[final_combined_data['batsman_runs']=='NaN']

##### also remove the inning columns since hence we use entire data for both 1st and 2nd innings.

del featured_2018_data['inning']

del featured_2018_data['batsman_runs']

In [321]:
##### now seperating the 1st and 2nd innnings data for the remaining data records.

featured_1st_inn_train = final_combined_data[final_combined_data['inning']==1]
##### removing inning column.

del featured_1st_inn_train['inning']


featured_2nd_inn_train = final_combined_data[final_combined_data['inning']==2]
##### removing inning column.

del featured_2nd_inn_train['inning']



In [326]:
##### splitting target variables and predictor variables.


y_train_1st_inn=featured_1st_inn_train.values[:,1]


X_train_1st_inn = featured_1st_inn_train

del X_train_1st_inn['match_id']
del X_train_1st_inn['batsman_runs']

In [327]:
X_train_1st_inn

,venue_id_1,venue_id_10,venue_id_11,venue_id_12,venue_id_13,venue_id_14,venue_id_15,venue_id_16,venue_id_17,venue_id_18,...,685,686,687,688,689,690,691,692,693,696
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [323]:
##### Lets try implementing gradient boosting.
##### Metrics for testing accuracy is Mean Square Error.

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import train_test_split



In [332]:
X_train,X_test,y_train,y_test=train_test_split(X_train_1st_inn,y_train_1st_inn,test_size=0.4,random_state=100)

In [336]:
est = GradientBoostingRegressor(n_estimators=100,
                                learning_rate=0.5,
                                max_depth=1,
                                random_state=0,
                                loss='ls').fit(X_train, y_train)

In [339]:
predict_y=est.predict(X_test)

In [335]:
y_test

array([26, 4, 32, ..., 8, 39, 33], dtype=object)

In [340]:
mean_squared_error(predict_y,y_test)

463.77320581498492

In [341]:
est.set_params(n_estimators=200, warm_start=True)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.5, loss='ls', max_depth=1, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=200, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=True)

In [342]:
est.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.5, loss='ls', max_depth=1, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=200, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=True)

In [344]:
predict_y=est.predict(X_test)

In [345]:
mean_squared_error(y_test,predict_y)

519.01869731736292

In [346]:
X_train

,venue_id_1,venue_id_10,venue_id_11,venue_id_12,venue_id_13,venue_id_14,venue_id_15,venue_id_16,venue_id_17,venue_id_18,...,685,686,687,688,689,690,691,692,693,696
2747,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
218,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3538,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2917,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3551,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
467,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1380,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4612,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#####################################################################################################################

In [347]:
#### gradient descent for the optimization.

X_train.shape

(1579, 234)

In [359]:
##### no. of variables is 234

##### so weights for 234 variables.

weight=np.zeros(shape=(1,234))
intercept=0



In [356]:
X_train_array=(np.array(X_train))

In [367]:
y=(np.dot(X_train_array,weight.T))+intercept   ##### while adding the intercept.

##### for each optimization i want to update the 


In [368]:
y

array([[ 0.],
       [ 0.],
       [ 0.],
       ..., 
       [ 0.],
       [ 0.],
       [ 0.]])

In [381]:
X_train_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [406]:
np.dot((((np.dot(X_train_array,weight.T))+intercept)-y_train).T,X_train_array).shape

(1, 234)

In [387]:
X_train_array.shape

(1579, 234)

In [394]:
y_train=y_train.reshape(1579,1)

In [419]:
##### Hyper parameter alpha = 0.05

gradient=((np.dot((((np.dot(X_train_array,weight.T))+intercept)-y_train).T,X_train_array))/1579)*0.05

In [417]:
gradient_intercept=np.sum((((np.dot(X_train_array,weight.T))+intercept)-y_train)/1579)*0.05

In [418]:
weight.shape

(1, 234)

In [421]:
(weight-gradient).shape

(1, 234)

In [427]:
#### cost function calculation.


### 1/2m(sum((h(x)-y)^2)).
### h(x)=x.w.T+intercept

(np.sum((((np.dot(X_train_array,weight.T))+intercept)-y_train)**2))/(2*1579)




496.76979100696644

In [454]:
##### number of epochs 1000 iterations.

cost = list()
weight=np.zeros(shape=(1,234))
intercept=0

for epochs in range(2000):
    
    ##### gradient for all the parameters except x0.
    gradient=((np.dot((((np.dot(X_train_array,weight.T))+intercept)-y_train).T,X_train_array))/1579)
    
    
    #### gradient for the only intercept x0.
    gradient_intercept=np.sum((((np.dot(X_train_array,weight.T))+intercept)-y_train)/1579)
    
    weight=weight-(gradient)*1
    
    intercept = intercept-(gradient_intercept)*1
    
    ##### cost function.
    
    cost.append((np.sum((((np.dot(X_train_array,weight.T))+intercept)-y_train)**2))/(2*1579))
    
    

In [455]:
cost

[261.28929566731506,
 251.25774103213413,
 249.4807383490389,
 248.15923309090076,
 246.97825347344732,
 245.90009005748726,
 244.90414486663053,
 243.9755241767119,
 243.10315453816966,
 242.27868524441,
 241.49572685834664,
 240.74931490645267,
 240.03552977037913,
 239.35122559033977,
 238.69383561808664,
 238.06123145679058,
 237.45162048987027,
 236.86347052956626,
 236.29545398502844,
 235.74640611818913,
 235.2152935361733,
 234.7011901748357,
 234.20325880522444,
 233.7207366435303,
 233.2529240345525,
 232.7991754563923,
 232.3588922933072,
 231.93151696723845,
 231.51652812270427,
 231.1134366356951,
 230.7217822730432,
 230.34113086988557,
 229.97107192351592,
 229.6112165248399,
 229.261195565903,
 228.92065817510763,
 228.58927034171975,
 228.26671369897633,
 227.95268444109897,
 227.64689235413684,
 227.34905994425642,
 227.05892164997923,
 226.77622312718353,
 226.50072059752645,
 226.23218025244694,
 225.9703777061442,
 225.71509749188536,
 225.46613259683758,
 225.2232

In [456]:
cost[1999]

209.84491946914756

In [457]:
weight

array([[4.8455493059833845, -3.9821947458791342, -5.817455712919996,
        0.7365521827833231, -5.318056673805346, 13.123583068679403,
        1.7733718556090499, -2.129658281021788, -18.91474673885955,
        0.04688578528401237, 8.934781249079016, 5.169784617488278,
        -2.7200892960534895, -0.532843705320452, 5.965584407799746,
        -3.0827052296897466, -0.9761543248127535, 1.3212633665554723,
        -1.067932739271599, -2.924193363361856, 1.1345483765912736,
        -4.5632165646929685, 2.9641721947523014, -0.3859125865618879,
        -6.290770755440672, 6.91947307128861, 8.888849359089928,
        3.2722336404300787, 4.002846867478615, 1.4357779328256859,
        -1.123354866974131, 2.8078865918594893, -2.444862926545466,
        2.9023250134934595, 6.441708040893393, 1.8390639455115554,
        2.1955745976790664, 6.327269997640909, -0.18605783140263155,
        -2.087224109180533, -4.341847817462196, -0.2276717776889978,
        3.7583591118775317, 0.166232725181447, 

In [458]:
from sklearn.linear_model import LinearRegression

In [461]:
LinearRegression.fit(X_train_array, y_train)

TypeError: fit() missing 1 required positional argument: 'y'